In [1]:
import numpy as np
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 32
# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [3]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)



Found 1174 images belonging to 14 classes.
Found 428 images belonging to 14 classes.


In [5]:
# Define the VGG16 base model
def create_InceptionV3_model():
    InceptionV3_model = InceptionV3(
        include_top=False, 
        input_shape=(img_height, img_width, 3)
        )
    InceptionV3_model.trainable = False

    model = Sequential([
        InceptionV3_model,
        GlobalAveragePooling2D(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        Dense(14, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    print(model.summary())
    return model




In [6]:
#MODEL fit
model = create_InceptionV3_model()
model.fit(train_generator, epochs=3, validation_data=test_generator)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2, 2, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 batch_normalization_94 (Bat  (None, 64)               256       
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 14)                9

In [7]:
# predicting train dataset 
train_pridiction = model.predict(train_generator)

# true labels 
train_true_labels = train_generator.classes




37/37 [==============================] - 18s 365ms/step


In [8]:
print( train_pridiction.shape)
print( train_true_labels.shape)

#adding the true labels to the pridiction
train_pridiction = np.concatenate((train_pridiction, train_true_labels.reshape(-1,1)), axis=1)
print( train_pridiction.shape)


(1174, 14)
(1174,)
(1174, 15)
